# Analyzing velocity and pumping with Pharaglow

### Importing packages

In [1]:
import pandas as pd
import os
import numpy as np
import sympy as sy

# plotting
import matplotlib.pylab as plt

# image processing
import skimage
from skimage import io
from skimage.util import img_as_float
from skimage.filters import threshold_li
from scipy import ndimage
from skimage.morphology import remove_small_objects
from skimage.exposure import histogram
from pharaglow.util import smooth

### Defining function to prepare binary of lawn image

In [2]:
def prepLawn(filename):
    # outlining lawn and turning into binary - thresholding with li (CHANGE NAME)
    image = img_as_float(skimage.io.imread(filename)).astype(np.float64)
    thresh = threshold_li(image)
    binary = image > thresh
    # troubleshooting images with noise around the lawn (optional)
    binary = remove_small_objects(binary, min_size = 5000, connectivity = 2) #change min_size 
    # filling small holes in the lawn
    binary = ndimage.binary_fill_holes(binary).astype(int)
    return binary

### Directing path options and video fps (CHANGE NAMES OR YOUR FILES ARE OVERWRITTEN)

In [3]:
# CHANGE NAME TO FOLDER OF INTEREST
pathname = "/media/scholz_la/hd2/Nicolina/Pharaglow/Pharaglow_v5/10x_INF100/"
outfile_vs = "/media/scholz_la/hd2/Nicolina/Nicolina/Saved_cutVelocities/10x_INF100.5_Velocities.csv"
outfile_cutv = "/media/scholz_la/hd2/Nicolina/Nicolina/Saved_cutVelocities/10x_INF100.5_cutVelocities.csv"

# IS THERE A LAWN?
lawn = 'Yes' #or 'No'
print('Is there a lawn?', lawn)

# FPS
frames = 30
print('fps:', frames)

# CHANGE FIGURE NAMES
all_velocities = "/home/nzjacic/Desktop/Rough_figures/10X_INF100/10x_INF100.5_all_velocities.pdf"
averaged_velocities = "/home/nzjacic/Desktop/Rough_figures/10x_INF100/10x_INF100.5_combined_averaged_velocities.pdf"
entry_trajectory = "/home/nzjacic/Desktop/Rough_figures/10X_INF100/10x_INF100.5_entries_trajectories.pdf"

Is there a lawn? Yes
fps: 30


### Making a dictionary of lawn images based on experiment ID (NZxxxx)

In [4]:
lawnImages = {}
if lawn=='Yes':
    for filename in os.listdir(pathname):
        if filename.endswith(".json"):
            # checking file has been imported
            print(filename)
            # get lawn picture
            lawnfile = filename.split('_')[1]
            if lawnfile not in lawnImages.keys():
                lawnpath = '/media/scholz_la/hd2/Nicolina/Lawns/'
                lawnImages[lawnfile] = prepLawn(os.path.join(lawnpath, lawnfile+'_lawn.tiff') or os.path.join(lawnpath, lawnfile+'_lawn.bmp'))

20200326_NZ0046_test_results_0.json
20200326_NZ0046_test_results_1.json
20200326_NZ0046_test_results_3.json
20200326_NZ0046_test_results_4.json
20200326_NZ0046_test_results_5.json
20200326_NZ0046_test_results_6.json
20200326_NZ0046_test_results_7.json
20200326_NZ0046_test_results_8.json
20200326_NZ0046_test_results_9.json


### Importing and filtering pharaglow data for "true" trajectories and lawn entries

In [5]:
%%time
# lists
# velocities = []
t0s = []
xvalues = []
yvalues = []
expIds = []
insides = []
# loop for importing and saving information from a single results folder into lists
for filename in os.listdir(pathname):
    if filename.endswith(".json"):
        # checking file has been imported
        print(filename)
        # getting python to read .json files
        data = pd.read_json(os.path.join(pathname, filename), orient='split', numpy=True)
        # data extraction if there is a lawn
        if lawn == 'Yes':
            # extracting useful variables
            x = data['x'].values
            y = data['y'].values
            f = data['frame'].values
            z = data['inside'].values
            # filter worms out that are mostly in the lawn at all times
            print("mean(z):", np.mean(z))
            if np.mean(z)<0.1:
                print("Rejected")
                continue
            if np.mean(z)>0.9:
                print("Rejected")
                continue
            # find first point of entry for all worms where t0=/=0 (i.e. where worm is already inside the lawn at 
            # the beginning of the trajectory)
            try:
                # t0 = first point where the worm is inside the lawn
                t0 = np.where(z==1)[0][0]
                print("t0:", t0)
                if t0==0:
                    print("Rejected")
                    continue
                t0s.append(t0)
                insides.append(z)
                xvalues.append(x)
                yvalues.append(y)
                expIds.append(filename.split('_')[1])
            except IndexError:
                print('Rejected')
                continue
            print("t0s:", t0s)
            print("Accepted")
        # data extraction if it's a no lawn control
        else:
            # velocity calculations
            x = data['x'].values
            y = data['y'].values
            t = data['frame'].values

20200326_NZ0046_test_results_0.json


/home/nzjacic/.conda/envs/pumping/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: the 'numpy' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'numpy'
  


mean(z): 0.5883987109678853
t0: 1413
t0s: [1413]
Accepted
20200326_NZ0046_test_results_1.json
mean(z): 0.0
Rejected
20200326_NZ0046_test_results_3.json
mean(z): 0.6122868545168703
t0: 1446
t0s: [1413, 1446]
Accepted
20200326_NZ0046_test_results_4.json
mean(z): 0.26752633490737376
t0: 4033
t0s: [1413, 1446, 4033]
Accepted
20200326_NZ0046_test_results_5.json
mean(z): 0.1369435042957563
t0: 3218
t0s: [1413, 1446, 4033, 3218]
Accepted
20200326_NZ0046_test_results_6.json
mean(z): 0.0
Rejected
20200326_NZ0046_test_results_7.json
mean(z): 0.0
Rejected
20200326_NZ0046_test_results_8.json
mean(z): 0.012641815235008104
Rejected
20200326_NZ0046_test_results_9.json
mean(z): 0.6129032258064516
t0: 0
Rejected
CPU times: user 1min 7s, sys: 5.23 s, total: 1min 13s
Wall time: 1min 12s


## Velocity

In [23]:
# defining lists
vs = []
cutv = []

# cut is the number of frames on either side of t0 we want to see
cut = 1000

# #converting time from frames to seconds
# time = np.linspace(0,2*cut/30., 2*cut)
df = np.diff(f)
print('checkpoint')

# with lawn
plt.figure(figsize=(12,8))
print('checkpoint')

for pid in data['particle'].unique():
    if lawn == 'Yes':
        print('checkpoint')
#         df = np.diff(f)
        t0 = t0s[pid]
        try:
            if t0>cut:
                print('pid:', pid)
                #converting time from frames to seconds
                time = np.linspace(0,2*cut/30., 2*cut)
                #velocity calculations + conversion from units/frame to microns/s
                v = np.sqrt((np.diff(x)**2+np.diff(y)**2))/f*30*2.349
                vs.append(v)
                #checking length of velocity column (optional)
                print('v.iloc:', v.iloc[t0-dt:t0+dt])
                #plotting velocity graph
                plt.plot(time, v.iloc[t0-cut:t0+cut], 'navy', alpha=0.1)
                #saving section of velocity around t0
                cutv.append(v.iloc[t0-cut:t0+cut].values)
            else:
                print('trajectory is too short')
        except ValueError:
            print('t0-dt or t0+dt exceeds number of frames')
            continue      
        #STDV and average
        plt.plot(time, np.mean(np.array(vcut), axis=0), color='navy')
        plt.plot(time, util.smooth(np.mean(np.array(vcut), axis=0), 30), color='r')
        #t0 line
        plt.axvline(dt/30, color='k', linestyle='--')
        #labeling axes and setting limits
        plt.ylabel(r"velocity ($\mu$m/s)");
        plt.xlabel("time (s)");
        plt.ylim(0,150)
        # save plot as .pdf
        plt.savefig(all_velocities)
    
    # without lawn
    else:
        # plotting so that trajectories all begin when t = 0s on graph
        plt.plot(time[:-1], v, color = 'b', alpha = 0.3)
        plt.ylabel('Velocity (microns/second)')
        plt.xlabel('Time (seconds)')
        # making sure all graphs are on the same y-scale to be more comparable
        plt.ylim(top = 300.0, bottom = 0.0)
        # make the trajectories the same length - look at the figure and change "cut" value based on data
        vn = np.pad(v[:cut], [0, cut-len(v[:cut])], mode='constant', constant_values=np.nan)
        cutVelocity.append(vn)
        # save plot as .pdf
        plt.savefig(all_velocities)

checkpoint
checkpoint
checkpoint


IndexError: list index out of range

<Figure size 864x576 with 0 Axes>

### Saving velocity data

In [ ]:
# making velocities and cutVelocities into a data frame
features_vs, features_cutv = vs, cutv
features_vs, features_cutv = pd.DataFrame(features_vs), pd.DataFrame(features_cutv)
print("Checking velocity:", features_vs, "Checking cut velocities:", features_cutv)

#checking data frame
features_vs.info(memory_usage='deep')
features_cutv.info(memory_usage='deep')

#saving as .csv file
features_vs.to_csv(outfile_vs)
features_cutv.to_csv(outfile_cutv)